In [ ]:
import base64
import openai

openai.api_key = "my-api-key"

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

image_path = "question-1.jpg" # Path to your image
base64_image = encode_image(image_path) # Getting the base64 string

math_topic = "algebra"
promt = f"""Ignore previous prompts. 
The math problem in the image is related to {math_topic}. Analyze the problem as if you are a mathematics teacher specializing in {math_topic}. Your task is to solve the problem in a clear, step-by-step manner with accuracy and a friendly tone like starting the sentence with "Lets" in step-by-step section. Follow structured format below in your response: 
-Problem: Restate the math problem briefly. 
-Step-by-Step Solution: Break the solution into clear and logical steps. Explain each step in simple, conversational language as if solving together. 
-Result: Present the final result in a clear format labeled as “Result:”. 
"""

response = openai.ChatCompletion.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": promt,
        },
        {
          "type": "image_url",
          "image_url": {
            "url":  f"data:image/jpeg;base64,{base64_image}"
          },
        },
      ],
    }
  ],
  temperature = 0,
)

print(response.choices[0])

In [2]:
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
def ocr_math_problem(image_path):
    try:
        # Görüntüyü aç
        image = Image.open(image_path)

        # OCR işlemini uygula
        text = pytesseract.image_to_string(image, lang='eng')  # Türkçe için 'tur' eklenebilir

        return text.strip()
    except Exception as e:
        return f"Hata: {e}"
# Örnek kullanım
image_path = "input images/img6.jpg"  # Görüntü dosyanızın yolu
math_problem = ocr_math_problem(image_path)
print("Tespit Edilen Matematik Sorusu:")
print(math_problem)


Tespit Edilen Matematik Sorusu:
2x-3=-7


In [3]:
import re
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

In [4]:
#DETECT THE MATH TOPIC
def which_math_topic(user_input_par):
    #Simple Algebra that can be solved by Python easly
     #if input text is just one line(for first version of program)
     #if input text doesn't contain any letter("x" could mean multiply but lets ignore for now)
    if("\n" not in user_input_par) and (re.search('[a-zA-Z]', user_input_par) == None):
        return "basic algebra"
    elif("\n" not in user_input_par) and (re.search('[a-zA-Z]', user_input_par)): # ex: 2x-3=-7
        return "algebra equations"
    else:
        return "have to send prompt"
which_math_topic(math_problem)

'algebra equations'

In [5]:
def basic_algebra_solving_1(question_par):
    if(question_par[-1] == "="): #make it compatible with Python
        question_par = question_par[:-1] # "5+5=" -> "5+5"

    question_par = question_par.replace("÷","/") # "10÷2" -> "10/2"
    question_par = question_par.replace(".","*") # "5.2" -> "5*2" #lets ignore decimal numbers for now

    #DETECT INVISIBLE MULTIPLICATION "5(1+2)" -> "5*(1+2)"
    for index, x in enumerate(question_par): #detect invisible multiplication sign and convert to Python syntax
        if(x == "(") and (index -1 >= 0): #if "(" character is not at the beginning
            if(question_par[index-1].isdigit()): #if left of the "(" character is number
                question_par = question_par[:index] + "*" + question_par[index:] # "5(2+1)" -> "5*(2+1)"
                #print("Değişti", question_par)
    
    for index, x in enumerate(question_par): #detect invisible multiplication sign and convert to Python syntax
        if(x == ")") and (index+1 < len(question_par)):
            if(question_par[index +1].isdigit()):
                question_par = question_par[:index+1] + "*" + question_par[index+1:] # "5+(2+1)3" -> "5+(2+1)*3"
                #print("Değişti", question_par)

    try:
        return question_par+ " math problem's result is:\n"+ str(eval(question_par))
    except:
        return "hata" #burada chatgpt apisine gidecek

def equations_algebra_solving_2(question_par): #"3x+10=-8" #"2x-3=-7"
    #OPEN DRIVER
    options = webdriver.ChromeOptions() # WebDriver ayarları (örneğin, ChromeDriver kullanılıyor)
    # DevTools'u etkinleştir
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)
    #options.add_argument("--headless")  # Tarayıcıyı başlıksız çalıştırmak için
    driver = webdriver.Chrome(options=options)
    driver.execute_cdp_cmd('Network.setCacheDisabled', {'cacheDisabled': True})# Cache'i devre dışı bırak
    #---
    #GET SOLVING OF THE EQUATION
    question_par = question_par.replace("=", "%3D") # for url, we have to convert "=" letter to "%3D" string
    driver.get("https://www.symbolab.com/solver/equation-calculator/" + question_par+ "?or=input")
    source = driver.page_source
    soup = BeautifulSoup(source, "html.parser")
    driver.quit() #QUIT DRIVER
    #---
    #PRINT STEPS AND RESULT
    try:
        result_text_list = []
        text_index = 1
        step_tags = soup.find("div",{"id":"solution-steps-target"}).find("div",{"class":"solution_step_list"})
        for index, x in enumerate(step_tags):
            if(x['class'] == ["interim_container"]): # add order to 
                #print(str(text_index)+".", x.text+":") #"Move 3 to the right side" -> "1. Move 3 to the right side "
                result_text_list.append(str(text_index)+". "+ x.text+":")
                text_index +=1
            elif(index+1 == len(step_tags)): #get last element of tag
                #print("Result is:", x.text)
                result_text_list.append("Result is: "+x.text)
            elif(index == 0):
                result_text_list.append("Problem is: "+x.text)
            else:
                result_text_list.append(x.text)
        return "\n".join(result_text_list)
    except:
        return "hata"

In [6]:
#MAIN
result_topic = which_math_topic(math_problem)
#BASIC ALGEBRA THAT CAN BE EASLY SOLVED BY PYTHON
if result_topic == "basic algebra":
    result = basic_algebra_solving_1(math_problem)
elif result_topic == "algebra equations":
    result = equations_algebra_solving_2(math_problem)
else: #send to gpt api
    pass
print(result)

Problem is: 2x−3=−7
1. Move 3 to the right side :
2x=−4
2. Divide both sides by 2 :
Result is: x=−2
